# index
- https://python.langchain.com/docs/tutorials/rag/#next-steps

# Setting

In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [12]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print((torch.cuda._get_nvml_device_index(0)))

True
NVIDIA A100-SXM4-80GB
3


# datasets

In [3]:
import dask.dataframe as dd
import pandas as pd

In [2]:
from datasets import load_dataset

In [3]:
wiki_dataset = load_dataset("../data/wikipedia-korean-20240501-1million-qna/data")

In [5]:
wiki_dataset["train"][0]

{'question': "나혜석이 1930년대 신문삽화 '섣달대목'에서 명절이 여성들에게 고단한 날임을 지적한 이유는 무엇인가요?",
 'answer': '나혜석은 명절이 여성들에게 고단한 날임을 지적한 것은 그들이 일상과 가사노동에 치여 눈코 뜰 새 없이 분주한 섣달의 풍경을 담고 있으며, 계속해서 신문과 잡지에 발표하는 만평형식의 목판화에도 신, 구 여성의 고달픈 일상에 대한 연민을 나타냈다고 생각했다. 또한 그는 명절이 여자들에게만 일을 시키는 고통스러운 날이라고 지적했다.',
 'context': "성들의 일상과 가사노동을 중심으로 눈코 뜰 새 없이 분주한 섣달의 풍경을 담고 있으며, 계속해서 신문과 잡지에 발표하는 만평형식의 목판화에도 신, 구 여성의 고달픈 일상에 대한 연민 을 나타냈다.\n또한 그는 명절이 여자들에게만 일을 시키는 고통스러운 날이라고 지적했다. 나혜석이 1930년대 신문삽화 '섣달대목'으로 일찌감치 명절이 여성들에게 고단한 날임을 고발하였다. 그가 명절의 고단함을 지적한 것은 후일 '명절 증후군'이라는 이름으로 사회적 화두가 되기도 했다.\n=== 사회 개혁론 ===\n그는 유럽 여행을 마치고 귀국한 후 여행기 ‘구미유기’에서 영국 참정권 운동을 소개하였다. 개화파의 실패 이후 참정권에 거부반응을 보이던 백성들을 향해, 국민이 정치에 참여하는 것은 당연한 것이라며 민주주의와 참정권의 당위성을 역설하였다. 그러나 정치인이나 정부를 양반의 연장으로 보고, 상전처럼 여기던 당시의 백성들은 그의 참정권 주장을 이상하게 여겼으며, 개화당의 아류, 여자 개화당 정도로 취급하며 무시하였다.\n영국의 참정권 운동을 소개하면서 참여한 영국여성운동가의 활약을 알렸다. 인간평등에 기초한 참정권운동뿐만 아니라 노동, 정조, 이혼, 산아제한, 시험결혼 등 여성문제를 소개하였다. 이후 언론과 칼럼, 강연을 통해 노동 문제, 임금 인상, 해고되지 않을 권리, 정당한 노동 등의 문제를 다루었고, 정조 문제, 결혼의 부작용을 줄일 수 있는 동거혼 등에 대해서도 

In [8]:
qna_dataset = load_dataset("json", data_files={"train": "../data/qna_data/train.json"})

In [3]:
qna_dataset["train"][0]

{'도메인': '금융/보험',
 '카테고리': '사고 및 보상 문의',
 '대화셋일련번호': 'A4075',
 '화자': '고객',
 '문장번호': '1',
 '고객의도': '비밀번호 오류',
 '상담사의도': '',
 'QA': 'Q',
 '고객질문(요청)': '인터넷뱅킹 로그인이 안돼요?',
 '상담사질문(요청)': '',
 '고객답변': '',
 '상담사답변': '',
 '개체명 ': '인터넷뱅킹, 로그인',
 '용어사전': '인터넷뱅킹/ 금융서비스',
 '지식베이스': '로그인,금융서비스'}

In [ ]:
wiki_dataset["train"][-1]

In [9]:
from langchain_core.documents import Document

def preprocess_for_rag(raw_data: list) -> list:
    """
    raw_data: [{'도메인': ..., '카테고리': ..., '고객질문(요청)': ..., '상담사답변': ..., '개체명 ': ..., '지식베이스': ...}, ...]
    반환: Document(page_content, metadata) 리스트
    """
    processed_documents = []

    for item in raw_data:
        # 기본 context 구성
        context_parts = []
        if item.get("고객질문(요청)"):
            context_parts.append(item["고객질문(요청)"])
        if item.get("상담사답변"):  # 상담사 답변도 있을 경우 context에 추가
            context_parts.append(item["상담사답변"])

        # context 하나로 합치기
        context = "\n".join(context_parts).strip()

        # metadata 구성
        metadata = {
            "domain": item.get("도메인", ""),
            "category": item.get("카테고리", ""),
            "customer_intent": item.get("고객의도", ""),
            "agent_intent": item.get("상담사의도", ""),
            "entities": item.get("개체명 ", ""),    # 주의: 개체명 뒤에 공백 있음
            "glossary": item.get("용어사전", ""),
            "knowledge_base": item.get("지식베이스", ""),
            "qa_type": item.get("QA", ""),           # 'Q' 또는 'A' 구분
            "dialogue_id": item.get("대화셋일련번호", ""),
            "speaker": item.get("화자", "")
        }

        # Document 생성
        if context:  # context가 비어있지 않으면 추가
            doc = Document(page_content=context, metadata=metadata)
            processed_documents.append(doc)

    return processed_documents


In [13]:
qna_dataset['train']

Dataset({
    features: ['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA', '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명 ', '용어사전', '지식베이스'],
    num_rows: 1782303
})

In [24]:
# 'train' split 선택
train_data = qna_dataset['train']

# 리스트 of dict 변환
try:
    raw_data = train_data.to_list()  # 최신 datasets 버전
except:
    raw_data = train_data.to_pandas().to_dict(orient="records")  # 범용

# 이제 전처리
docs_for_rag = preprocess_for_rag(raw_data)

# # 결과 확인
# for doc in docs_for_rag:
#     print(doc.page_content)
#     print(doc.metadata)

# Load

In [9]:
from langchain_community.document_loaders import (
    CSVLoader,
    PyPDFLoader, 
    Docx2txtLoader,
    TextLoader,
    JSONLoader,
)

In [ ]:
wiki_docs = JSONLoader("../data/wikipedia-korean-20240501-1million-qna/data", jq_schema='.', text_content=False).load()       

In [10]:
qna_docs = JSONLoader("../data/qna_data/train.json", jq_schema='.', text_content=False).load()       

# Spilt

In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveJsonSplitter

In [26]:
model_name = "../ai_models/base_models/BGE-m3-ko"
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/tmp/ipykernel_3820349/4211131713.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [27]:
# splitter = RecursiveJsonSplitter(max_chunk_size=512)

In [ ]:
# json_chunks = splitter.split_json(json_data=docs_for_rag)

In [32]:
# 한국어 문장 경계를 잘 인식하는 청커
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,      # 512 tokens 기준 (한글 약 400~500자)
    chunk_overlap=80,    # 문맥 이어지게 약 80 tokens 겹치기
    separators=["\n\n", "。", ".", "!", "?", "\n"]  # 문장 경계 세밀하게 설정
)

# 검색된 큰 문단을 문장 기준으로 late chunking
chunks = text_splitter.split_documents(docs_for_rag)


# Store

In [35]:
from langchain.vectorstores import FAISS

# docs_for_rag은 이미 Document 리스트 상태
vector_store = FAISS.from_documents(
    documents=docs_for_rag,
    embedding=embeddings
)

# 저장 (로컬 디렉토리 지정)
vector_store.save_local("faiss_index")


In [36]:
import json

def save_docs_as_jsonl(documents, output_path):
    """
    documents: List[Document]
    output_path: 저장할 파일 경로 (예: 'output.jsonl')
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for doc in documents:
            json_obj = {
                "page_content": doc.page_content,
                "metadata": doc.metadata
            }
            f.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

# 사용 예시
save_docs_as_jsonl(docs_for_rag, "../data/qna_data/docs_for_rag.jsonl")
